In [2]:
# import libraries

import dlib
import cv2
import numpy as np
from scipy.spatial import distance as dist

In [43]:
cap.release()
cv2.destroyAllWindows()

In [3]:
# function for EAR

def eye_aspect_ratio(eye):
    # euclidean distances between the two sets of vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    # euclidean distance between the horizontal eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])
    # eye aspect ratio
    ear = (A + B) / (2.0 * C)
    return ear

In [4]:
# function for MAR

def mouth_aspect_ratio(mouth):
    # euclidean distances between the two sets of vertical mouth landmarks (x, y)-coordinates
    A = dist.euclidean(mouth[2], mouth[10])
    B = dist.euclidean(mouth[4], mouth[8])
    # euclidean distance between the horizontal mouth landmark (x, y)-coordinates
    C = dist.euclidean(mouth[0], mouth[6])
    # mouth aspect ratio
    mar = (A + B) / (2.0 * C)
    return mar

In [53]:
# initialize detector and predictor
faces_bboxes = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor("../models/dlib_shape_predictor_68_face_landmarks.dat")

# initialize counters and threshold
frame_counter = 0
ear_frame_counter = 0
ear_threshold = []
hc_ear_threshold = 0.25

# open videocapture from the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # update frame counter
    frame_counter += 1
    # convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # find faces
    faces = faces_bboxes(gray, 0)

    # initialize empty array with shape (68, 2) (the predictor return 68 facemarks x y coordinates)
    coords = np.zeros((68, 2), dtype="int") # need to specify the datatype for convexHull to work

    # loop over the bounding box
    for face in faces:
        # find landmark for every bounding box
        landmarks = landmark_predictor(gray, face)
        # loop over landmarks to update the coordinates array
        for i in range(0, landmarks.num_parts):
            coords[i] = (landmarks.part(i).x, landmarks.part(i).y)

    # slice coords to find landmarks of eyes and mouth
    left_eye = coords[42:48]
    right_eye = coords[36:42]
    mouth = coords[48:68]
    
    # find left and right EAR
    left_ear = eye_aspect_ratio(left_eye)
    right_ear = eye_aspect_ratio(right_eye)
    
    # average of both EARs
    ear = (left_ear + right_ear) / 2

    # find MAR
    mar = mouth_aspect_ratio(mouth)

    # find convex hull
    leye_hull = cv2.convexHull(left_eye)
    reye_hull = cv2.convexHull(right_eye)
    mouth_hull = cv2.convexHull(mouth)
    
    # draw eye and mouth contours
    cv2.drawContours(frame, [leye_hull], -1, (0, 255, 0), 2)
    cv2.drawContours(frame, [reye_hull], -1, (0, 255, 0), 2)
    cv2.drawContours(frame, [mouth_hull], -1, (0, 255, 0), 2)
    
    # take some time to initialize values
    if frame_counter < 30:
        cv2.putText(frame, "CALIBRATING", (20, 20), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255))
        ear_threshold.append(ear)
        print(frame_counter)
    else:
        # convert ear_threshold to numpy array
        ear_threshold = np.array(ear_threshold)
       # remove NaN values from the array
        ear_threshold = ear_threshold[~np.isnan(ear_threshold)] # ~ = is not
        # get the mean
        ear_threshold = np.mean(np.array(ear_threshold))
    
        # print EAR and MAR and get EAR threshold
        cv2.putText(frame, f"EAR: {round(ear, 2)}", (20, 20), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255))
        cv2.putText(frame, f"MAR: {round(mar, 2)}", (20, 50), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255))
        # check if ear goes below the threshold for a number of frames
        if ear < ear_threshold:
            ear_frame_counter += 1
            print(ear_frame_counter)

            if ear_frame_counter >= 10:
                print("EAR " + str(ear))
                print("THRESH" + str(ear_threshold))
                print("drowsy")
                cv2.putText(frame, "WARNING!", (20, 100), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255))

        else:
            ear_frame_counter = 0
        
    
    
    
    
    
    cv2.imshow('Webcam Feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

<ipython-input-3-12927eed7de8>:10: RuntimeWarning: invalid value encountered in double_scalars
  ear = (A + B) / (2.0 * C)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
1
1
2
1
1
1
1
1
2
3
4
1
2
3
1
2
1
1
1
2
1
2
1
2
3
4
5
6
7
1
2
3
4
5
6
7
8
9
10
EAR 0.25365180838308526
THRESH0.29231488826175533
drowsy
11
EAR 0.23567934500582216
THRESH0.29231488826175533
drowsy
12
EAR 0.24730401391799445
THRESH0.29231488826175533
drowsy
13
EAR 0.24722311506934605
THRESH0.29231488826175533
drowsy
14
EAR 0.2628280263702406
THRESH0.29231488826175533
drowsy
15
EAR 0.2782139156450485
THRESH0.29231488826175533
drowsy
16
EAR 0.2757341200745042
THRESH0.29231488826175533
drowsy
17
EAR 0.28339827261956263
THRESH0.29231488826175533
drowsy
18
EAR 0.2648589545834039
THRESH0.29231488826175533
drowsy
19
EAR 0.25483796372142187
THRESH0.29231488826175533
drowsy
20
EAR 0.24067766328065543
THRESH0.29231488826175533
drowsy
21
EAR 0.2690086030838222
THRESH0.29231488826175533
drowsy
22
EAR 0.23485791680048956
THRESH0.29231488826175533
drowsy
23
EAR 0.28518061928959015
THRESH0.29231488826175533
drowsy
24
EAR 0.